# Introduction

This is the demo how to use AI Bridge [Ben](https://github.com/lorserker/ben) and [redeal](https://github.com/anntzer/redeal) compare Leading for 3NTs

* http://www.rpbridge.net/4g00.htm
* book [Winning Notrump Leads](https://www.amazon.com/Winning-Notrump-Leads-David-Bird/dp/1554947596) - redeal uses the idea from this book
* AI Ben models: https://huggingface.co/AIBridgeEngine

# Preparation

both needs [dds](https://github.com/dds-bridge/dds) library, here gives the hacked solution without recompiling.

It shall work smoothly in [colab](https://colab.research.google.com/), verified in 2023.11.19, which is ubuntu jammy with sudo permission


In [ ]:
# install ai ben library & redeal library
!pip install --upgrade git+https://github.com/larrycai/ben-lite # install ben python package, not released, hacked
!pip install --upgrade git+https://github.com/anntzer/redeal

In [ ]:
!find ~/.local | grep redeal
!find ~/.local | grep ben
# !ldd ~/.local/lib/python3.10/site-packages/redeal/libdds.so
# may need to copy libdds.so to ben, this libdds remove mt-thread
#!cp ~/.local/lib/python3.10/site-packages/redeal/libdds.so !cp ~/.local/lib/python3.10/site-packages/ben/libdds.so

# Let's see the question

https://bridgeonline.ru/wp-content/uploads/2020/04/Winning_Notrump_Leads.pdf (googled, ok to use?)

Chapter 1, Hand 3

````
Bidding: (None all)
 West  North   East  South
                     1NT
 pass  3NT     all   pass

Sit in west: Which is the best lead

 South: 15-17 points, 5332, 4432, 4333
 North: 9-11 points, no 4 high cards
````

What would you lead from: ♠ J 8 6 ♥ 9 5 ♦ K Q 7 2 ♣ K J 7 2 ?

# Redeal's solution

Thomas Andrew's Deal is a deal generator: it outputs deals satisfying whatever conditions you specify, then estimate which is best by using dds

In [19]:
from redeal import *
from redeal import dds
from redeal.global_defs import Seat, Suit, Card, Rank, Strain
def accept(deal):
    north, south = deal.north, deal.south
    return (
        balanced(south) and       # south
        15 <= south.hcp <= 17 and # south
        len(north.spades) <4 and len(north.hearts) < 4 and # north
        9 <= north.hcp <= 11                               # north
    )

N_TRIES=500 # takes longer time if 5000, 500 is around
def nt_leader(predeal, del_lst=[], n_tries=N_TRIES):
    samples = []
    dealer = Deal.prepare(predeal)

    balanced = Shape("(4333)") + Shape("(4432)") + Shape("(5332)")

    simulation = OpeningLeadSim(accept, "3NS", imps)
    simulation.initial(dealer)

    found = 0
    for _ in range(1000*n_tries):
        if found >= n_tries:
            break
        deal = dealer()
        if not accept(deal):
            continue

        found += 1
        score_3NS = deal.dd_score('3NS')
        if score_3NS < 0:
            # it is down
            samples.append(deal)
        simulation.do(deal)

    # remove useless
    # check https://github.com/anntzer/redeal/blob/master/redeal/redeal.py#L623
    for card in del_lst:
        simulation.payoff.entries.remove(Card.from_str(card))

    print(f"Result of simulation: {found} deals")
    simulation.final(n_tries)
    return samples

predeal = {"W": "J86 95 KQ72 KJ72"}
print(predeal['W'])

cards_needs_removed=["SJ", "H5"]
samples = nt_leader(predeal,cards_needs_removed)
deal = samples[0]
Deal.set_str_style('long')
print("Check one sample")
print("3NT sample:", deal)

result=deal.dd_all_tricks("N" , "W")
score_3NS = deal.dd_score('3NS')


J86 95 KQ72 KJ72
Result of simulation: 500 deals
	♠8	♠6	♥9	♦K	♦7	♦2	♣K	♣J	♣7	♣2	
♠8		-0.25	-0.30	-0.02	-0.04	+0.85	+0.88	+0.87	+1.59	+1.32	+0.76	+0.68
		(0.07)	(0.08)	(0.16)	(0.16)	(0.18)	(0.18)	(0.18)	(0.18)	(0.18)	(0.18)	(0.17)
♠6	+0.25		-0.05	+0.22	+0.21	+1.08	+1.12	+1.11	+1.83	+1.56	+1.00	+0.92
	(0.07)		(0.03)	(0.15)	(0.14)	(0.18)	(0.18)	(0.18)	(0.18)	(0.18)	(0.17)	(0.16)
♥9	+0.30	+0.05		+0.27	+0.26	+1.14	+1.18	+1.16	+1.89	+1.61	+1.05	+0.97
	(0.08)	(0.03)		(0.14)	(0.14)	(0.18)	(0.17)	(0.17)	(0.18)	(0.18)	(0.16)	(0.16)
♦K	+0.02	-0.22	-0.27		-0.02	+0.87	+0.92	+0.90	+1.62	+1.35	+0.79	+0.71
	(0.16)	(0.15)	(0.14)		(0.02)	(0.17)	(0.16)	(0.16)	(0.17)	(0.15)	(0.13)	(0.13)
♦7	+0.04	-0.21	-0.26	+0.02		+0.89	+0.94	+0.92	+1.64	+1.37	+0.81	+0.73
	(0.16)	(0.14)	(0.14)	(0.02)		(0.17)	(0.16)	(0.16)	(0.17)	(0.15)	(0.13)	(0.13)
♦2	-0.85	-1.08	-1.14	-0.87	-0.89		+0.02	+0.00	+0.71	+0.43	-0.12	-0.20
	(0.18)	(0.18)	(0.18)	(0.17)	(0.17)		(0.12)	(0.12)	(0.19)	(0.19)	(0.18)	(0.18)
♣K	-0.88	-1.12	-1.18	-0.9

# AI Ben's solution

It uses AI Ben machine learning models to find the best leader


In [25]:
import huggingface_hub
import os
lead_id="AIBridgeEngine/Ben-3B-Lead-v0.1"
bidding_id="AIBridgeEngine/Ben-3B-Bidding-v0.1"
biddinginfo_id="AIBridgeEngine/Ben-3B-Biddinginformation-v0.1"
singledummy_id="AIBridgeEngine/Ben-3B-Singledummy-v0.1"
folder="hfmodels"
huggingface_hub.snapshot_download(repo_id=lead_id, local_dir=folder)
huggingface_hub.snapshot_download(repo_id=bidding_id, local_dir=folder)
huggingface_hub.snapshot_download(repo_id=biddinginfo_id, local_dir=folder)
huggingface_hub.snapshot_download(repo_id=singledummy_id, local_dir=folder)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

'/content/hfmodels'

In [26]:
from ben.nn.models import Models
from ben.bots import BotLead
from ben.sample import Sample
from ben.nn.leader import Leader
from ben.nn.bid_info import BidInfo
from ben.nn.bidder import Bidder
from ben.nn.lead_singledummy import LeadSingleDummy
lead = Leader(f'{folder}/lead-1000000')
biddinginfo = BidInfo(f'{folder}/binfo-100000')
bidder = Bidder("", f'{folder}/bidding-100000')
sd_model = LeadSingleDummy(f'{folder}/lr3-1000000')
models = Models(bidder, biddinginfo, lead, sd_model, None, 0.1, 0.05,0.75, 0.66)
sampler = Sample(0.01, 0.1, 0.03, 0.05, 64, 5000, 5000, 128, 100, False)


In [27]:
# both vulnerable. you are sitting North as dealer and you hold
hand = 'J86.950.KQ72.KJ72'

# the auction goes:
auction = ['PAD_START','PAD_START','PAD_START',"1N","PASS",'PASS', 'PASS', 'PASS']

# what to lead?

lead_bot = BotLead([True, True], hand, models, -1, -1, models.lead_threshold, sampler, False)
lead = lead_bot.find_opening_lead(auction)

Sorting by insta_score
Cx
0.4442762565795649


In [28]:
lead.card

S6

seems like the engine chose to lead the ace of diamonds

the other options it considered were: a small spade and a small club

In [29]:
lead.to_dict()['candidates']

[{'card': 'Sx',
  'insta_score': 0.168,
  'expected_tricks': 7.99,
  'p_make_contract': 0.8},
 {'card': 'Dx',
  'insta_score': 0.3827,
  'expected_tricks': 8.11,
  'p_make_contract': 0.84},
 {'card': 'Cx',
  'insta_score': 0.4443,
  'expected_tricks': 8.36,
  'p_make_contract': 0.86}]

in the above output:
- `insta_score` reflects the preference of the neural network
- `expected_tricks` how many tricks declarer is expected to take on this lead
- `p_make_contract` is the probability of the contract making on this lead

the probability of making and the expected tricks are computed on samples which are consistent with the auction. the samples are estimated single dummy using a neural network (more on this in another tutorial). we could also solve the samples double dummy, but that would be quite a bit slower.

In [30]:
# each row is one sample board
# the hands are in the order: LHO, Partner, RHO. Your cards are not shown as they are fixed/

lead.samples

['xxxx.x.Axx.A8xx T9x.AKT8xx.T8.xx AKQ.QJx.J9xx.QT9',
 'T9x.QTxx.Axx.T8x Qxxxx.AJ.8x.xxx AK.K8xx.JT9x.AQ9',
 'QTxx.J8x.9x.Q9xx 9xxx.AKT.J8x.8x AK.Qxxx.ATxx.ATx']